In [1]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn import utils
import csv
from tqdm import tqdm
import multiprocessing
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
import pandas as pd
import numpy as np

In [3]:
tqdm.pandas(desc="progress-bar")
# Function for tokenizing
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens
# Initializing the variables
train_documents = []
test_documents = []
'''i = 0
# Associating the tags(labels) with numbers
tags_index = {'sci-fi': 1 , 'action': 2, 'comedy': 3, 'fantasy': 4, 'animation': 5, 'romance': 6}
#Reading the file
#FILEPATH = 'tagged_plots_movielens.csv'
#with open(FILEPATH, 'r') as csvfile:
with open('tagged_plots_movielens.csv', 'r') as csvfile:
    moviereader = csv.reader(csvfile, delimiter=',', quotechar='"')
    for row in moviereader:
        if i == 0:
            i += 1
            continue
        i += 1
        if i <= 2000:
            train_documents.append(TaggedDocument(words=tokenize_text(row[2]), tags=[tags_index.get(row[3], 8)] ))
        else:
            test_documents.append( TaggedDocument(words=tokenize_text(row[2]),
 tags=[tags_index.get(row[3], 8)]))'''

categories = ['action_adventure','drama_romance','comedy_family','horror_thriller']
tmdbmovies_train = pd.read_csv("total_train.csv")
tmdbmovies_test = pd.read_csv("total_test.csv")
for i in range(len(tmdbmovies_train.data)):
    train_documents.append(TaggedDocument(words=tokenize_text(tmdbmovies_train["data"][i]),tags=[tmdbmovies_train["Label"][i]]))
for j in range(len(tmdbmovies_test.data)):
    test_documents.append(TaggedDocument(words=tokenize_text(tmdbmovies_test["data"][j]),tags=[tmdbmovies_test["Label"][j]]))

print(train_documents[0])


TaggedDocument(['the', 'clumsy', 'and', 'unfunny', 'clown', 'richard', '``', 'stitches', "''", 'grindle', 'entertains', 'at', 'the', '10th', 'birthday', 'party', 'of', 'little', 'tom', 'but', 'the', 'boy', 'and', 'his', 'friends', 'play', 'prank', 'with', 'stitches', 'tying', 'his', 'shoelaces', 'stitches', 'slips', 'falls', 'and', 'dies', 'six', 'years', 'later', 'tom', 'gives', 'birthday', 'party', 'for', 'his', 'friends', 'at', 'home', 'but', 'stitches', 'revives', 'to', 'haunt', 'the', 'teenagers', 'and', 'revenge', 'his', 'death'], [3])


In [5]:
model_dbow = Doc2Vec(dm=1, vector_size=300, negative=5, hs=0, min_count=2, sample = 0, alpha=0.025, min_alpha=0.001)
model_dbow.build_vocab([x for x in tqdm(train_documents)])
train_documents  = utils.shuffle(train_documents)
model_dbow.train(train_documents,total_examples=len(train_documents), epochs=30)
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors
model_dbow.save('./tmdbdoc2veclsa.d2v')
#model_dbow = Doc2Vec.load('./movieModel.d2v')
model_dbow

100%|██████████████████████████████████████████████████████████████████████████████████████| 2400/2400 [00:00<?, ?it/s]


In [6]:
def vector_for_learning(model, input_docs):
    sents = input_docs
    targets, feature_vectors = zip(*[(doc.tags[0], model.infer_vector(doc.words, steps=20)) for doc in sents])
    return targets, feature_vectors

y_train, X_train = vector_for_learning(model_dbow, train_documents)
y_test, X_test = vector_for_learning(model_dbow, test_documents)

from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer

svd = TruncatedSVD(100)
lsa = make_pipeline(svd, Normalizer(copy=False))

X_train_lsa = lsa.fit_transform(X_train)
X_test_lsa = lsa.transform(X_test)

X_train = X_train_lsa
X_test = X_test_lsa

In [7]:
knn_cos = KNeighborsClassifier(n_neighbors=5, metric='cosine')
knn_cos.fit(X_train, y_train)
print("=================Doc2Vec + LSA + cosine==============")
print("accuracy:",knn_cos.score(X_test,y_test))
from sklearn import metrics
from sklearn.metrics import confusion_matrix
# Classify the test vectors.
y_pred = knn_cos.predict(X_test)
print(classification_report(y_test, y_pred, target_names=categories))
cm = pd.DataFrame(confusion_matrix(y_test, y_pred),index=['action:true', 'romance:true','comedy:true','horror:true'],columns=['action:pred', 'romance:pred','comedy:pred','horror:pred'])
cw = cm.sum(axis=0)
cm["All"] = cm.sum(axis=1)
row_df = pd.DataFrame([cw],index=["All"])
cm = pd.concat([ cm,row_df])
cm["All"] = cm.sum(axis=1)
print("confusion matrix:\n",cm)
print("accuracy:",metrics.accuracy_score(y_test, y_pred)*100)

=================Doc2Vec + LSA + cosine==============
accuracy: 0.38875
                  precision    recall  f1-score   support

action_adventure       0.38      0.58      0.46       453
   drama_romance       0.41      0.45      0.43       446
   comedy_family       0.40      0.28      0.33       409
 horror_thriller       0.34      0.15      0.21       292

        accuracy                           0.39      1600
       macro avg       0.38      0.37      0.36      1600
    weighted avg       0.39      0.39      0.37      1600

confusion matrix:
               action:pred  romance:pred  comedy:pred  horror:pred     All
action:true           262            86           71           34   906.0
romance:true          139           199           71           37   892.0
comedy:true           148           127          116           18   818.0
horror:true           140            76           31           45   584.0
All                   689           488          289          134  1600.